<a href="https://colab.research.google.com/github/dhanushpachabhatla/My_LangChain_Playground/blob/main/langchian_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompts Templating for Gemini - LangChain

In [1]:
!pip install langchain langsmith --quiet

In [2]:
!pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 32.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [3]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = getpass("Enter LangSmith API Key: ")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "pr-husky-bran-20"

Enter LangSmith API Key: ··········


#Basic Prompting

We'll start by looking at the various parts of our prompt. For RAG use-cases we'll typically have three core components however this is _very_ use-cases dependant and can vary significantly. Nonetheless, for RAG we will typically see:

* **Rules for our LLM**: this part of the prompt sets up the behavior of our LLM, how it should approach responding to user queries, and simply providing as much information as possible about what we're wanting to do as possible. We typically place this within the _system prompt_ of an chat LLM.

* **Context**: this part is RAG-specific. The context refers to some _external information_ that we may have retrieved from a web search, database query, or often a _vector database_. This external information is the **R**etrieval **A**ugmentation part of **RA**G. For chat LLMs we'll typically place this inside the chat messages between the assistant and user.

* **Question**: this is the input from our user. In the vast majority of cases the question/query/user input will always be provided to the LLM (and typically through a _user message_). However, the format and location of this being provided often changes.

* **Answer**: this is the answer from our assistant, again this is _very_ typical and we'd expect this with every use-case.

The below is an example of how a RAG prompt may look:

```
Answer the question based on the context below,                 }
if you cannot answer the question using the                     }--->  (Rules) For Our Prompt
provided information answer with "I don't know"                 }

Context: Aurelio AI is an AI development studio                 }
focused on the fields of Natural Language Processing (NLP)      }
and information retrieval using modern tooling                  }--->   Context AI has
such as Large Language Models (LLMs),                           }
vector databases, and LangChain.                                }

Question: Does Aurelio AI do anything related to LangChain?     }--->   User Question

Answer:                                                         }--->   AI Answer
```

In [6]:
prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""

LangChain uses a `ChatPromptTemplate` object to format the various prompt types into a single list which will be passed to our LLM:

* manual usage of ChatPromptTemplate

In [10]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}"),
])

When we call the template it will expect us to provide two variables, the `context` and the `query`. Both of these variables are pulled from the strings we wrote, as LangChain interprets curly-bracket syntax (ie `{context}` and `{query}`) as indicating a dynamic variable that we expect to be inserted at query time. We can see that these variables have been picked up by our template object by viewing it's `input_variables` attribute:

In [11]:
prompt_template.input_variables

['context', 'query']

In [12]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

* Template usage of ChatPromptTemplate

In [13]:
from langchain.prompts import ( SystemMessagePromptTemplate ,HumanMessagePromptTemplate)

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt),
    HumanMessagePromptTemplate.from_template("{query}"),
])

prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

this is much similar to the first one

#Hitting LLM

In [4]:
import os
from getpass import getpass
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ['GOOGLE_API_KEY'] = getpass("Enter your GEMINI KEY")

llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature= 0.2)

Enter your GEMINI KEY··········


In [14]:
pipeline = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | prompt_template
    | llm
)

In [26]:
context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [16]:
pipeline.invoke({"query": query, "context": context})

AIMessage(content="Aurelio AI is an AI company that develops tooling for AI engineers, focusing on language AI.  They specialize in building AI agents and have expertise in information retrieval.  They've created open-source frameworks like Semantic Router and Semantic Chunkers, and offer an AI Platform with tools for building AI applications.  Additionally, they provide development services to help other organizations bring their AI technology to market.  They became LangChain Experts in September 2024.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--718e85fb-6454-4d8d-98bb-97d4fe3bffd1-0', usage_metadata={'input_tokens': 180, 'output_tokens': 95, 'total_tokens': 275, 'input_token_details': {'cache_read': 0}})

#Few shot Prompting

In [17]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

In [18]:
examples = [
    {"input": "Here is query #1", "output": "Here is the answer #1"},
    {"input": "Here is query #2", "output": "Here is the answer #2"},
    {"input": "Here is query #3", "output": "Here is the answer #3"},
]

In [21]:
from langchain.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
# here is the formatted prompt
print(few_shot_prompt.format())

Human: Here is query #1
AI: Here is the answer #1
Human: Here is query #2
AI: Here is the answer #2
Human: Here is query #3
AI: Here is the answer #3


* real example

In [27]:
new_system_prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Always answer in markdown format. When doing so please
provide headers, short summaries, follow with bullet
points, then conclude.

Context: {context}
"""

prompt_template.messages[0].prompt.template = new_system_prompt

out = pipeline.invoke({"query": query, "context": context}).content
print(out)

# Aurelio AI: An Overview

Aurelio AI is an artificial intelligence company specializing in language AI and building tools for AI engineers.

*   **Focus:** Language AI, AI agents, information retrieval.
*   **Open-Source Frameworks:** Semantic Router and Semantic Chunkers.
*   **AI Platform:** Provides tooling for AI engineers.
*   **Development Services:** Helps organizations bring AI technology to market.
*   **Expertise:** LangChain Experts (since September 2024).


# Conclusion

Aurelio AI offers a range of services and tools centered around language AI, from open-source frameworks to a comprehensive AI platform and development services.  Their expertise in LangChain further solidifies their position in the AI engineering landscape.


In [28]:
from IPython.display import display, Markdown

display(Markdown(out))

# Aurelio AI: An Overview

Aurelio AI is an artificial intelligence company specializing in language AI and building tools for AI engineers.

*   **Focus:** Language AI, AI agents, information retrieval.
*   **Open-Source Frameworks:** Semantic Router and Semantic Chunkers.
*   **AI Platform:** Provides tooling for AI engineers.
*   **Development Services:** Helps organizations bring AI technology to market.
*   **Expertise:** LangChain Experts (since September 2024).


# Conclusion

Aurelio AI offers a range of services and tools centered around language AI, from open-source frameworks to a comprehensive AI platform and development services.  Their expertise in LangChain further solidifies their position in the AI engineering landscape.

its not bad but what if i want it in some strict markdown pattern then i may use exmamples

In [29]:
examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

In [30]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [24]:
from IPython.display import display, Markdown

In [31]:
out = few_shot_prompt.format()
display(Markdown(out))

Human: Can you explain gravity?
AI: ## Gravity

Gravity is one of the fundamental forces in the universe.

### Discovery

* Gravity was first discovered by Sir Isaac Newton in the late 17th century.
* It was said that Newton theorized about gravity after seeing an apple fall from a tree.

### In General Relativity

* Gravity is described as the curvature of spacetime.
* The more massive an object is, the more it curves spacetime.
* This curvature is what causes objects to fall towards each other.

### Gravitons

* Gravitons are hypothetical particles that mediate the force of gravity.
* They have not yet been detected.

**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.


Human: What is the capital of France?
AI: ## France

The capital of France is Paris.

### Origins

* The name Paris comes from the Latin word "Parisini" which referred to a Celtic people living in the area.
* The Romans named the city Lutetia, which means "the place where the river turns".
* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.

**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.



* we can inlclude fewshotprompting in our prommpt template like below

In [32]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    few_shot_prompt,
    ("user", "{query}"),
])

In [33]:
pipeline = prompt_template | llm
out = pipeline.invoke({"query": query, "context": context}).content
display(Markdown(out))

## Aurelio AI: Building the Future of Language AI

Aurelio AI is an artificial intelligence company specializing in developing tools for AI engineers, with a particular focus on language AI.

### Key Activities:

* **Developing AI tooling:**  They create frameworks and platforms to assist in the building of AI applications.  Notable open-source projects include Semantic Router and Semantic Chunkers.
* **Providing an AI Platform:** This platform offers engineers a suite of tools to streamline their AI development processes.
* **Offering Development Services:** Aurelio AI helps other organizations bring their AI technologies to market through custom development services.
* **LangChain Expertise:** Recognized as LangChain Experts since September 2024, demonstrating a strong track record of building AI solutions within the LangChain ecosystem.
* **Strong Background in Information Retrieval and AI Agents:** The team possesses significant expertise in these areas, informing their development efforts.


**In short,** Aurelio AI empowers AI engineers with the tools and expertise needed to build cutting-edge language AI applications.

#Chain Of Though Prompting

Next, let's look at Chain of Thought `CoT`, a prompting technique that gets Large Language Models `LLMs` to "think" step-by-step. Instead of just giving an answer, CoT encourages the LLM to break a problem down into smaller, manageable steps. This process makes the model more likely to reach the correct solution and significantly reduces the chance of it making things up, or "hallucinating."

You don't need any special tools from LangChain to use CoT. The magic happens in the system prompt itself. We'll simply instruct the LLM to outline the problems, solve each one individually, and then present the final answer.

Ready to see the difference? Let's first test our LLM without this technique to see how it performs.

* we can add explicit CoT prompting to our system prompt to see if we can get a better result.

In [36]:
query = (
    "How many keystrokes are needed to type the numbers from 1 to 500?"
)

In [37]:
# Define the chain-of-thought prompt template
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  subproblems that need to be solved to answer the
  question.
- Solve each sub problem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to
  provide the final answer.
"""

cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", cot_system_prompt),
    ("user", "{query}"),
])

cot_pipeline = cot_prompt_template | llm

In [38]:
cot_result = cot_pipeline.invoke({"query": query}).content
display(Markdown(cot_result))

To determine the total number of keystrokes needed to type the numbers from 1 to 500, we need to solve the following subproblems:

**Subproblem 1: Count keystrokes for numbers with one digit (1-9)**

* There are 9 one-digit numbers.
* Each number requires 1 keystroke.
* Total keystrokes for one-digit numbers: 9 * 1 = 9 keystrokes

**Subproblem 2: Count keystrokes for numbers with two digits (10-99)**

* There are 90 two-digit numbers (99 - 10 + 1 = 90).
* Each number requires 2 keystrokes.
* Total keystrokes for two-digit numbers: 90 * 2 = 180 keystrokes

**Subproblem 3: Count keystrokes for numbers with three digits (100-500)**

* There are 401 three-digit numbers (500 - 100 + 1 = 401).
* Each number requires 3 keystrokes.
* Total keystrokes for three-digit numbers: 401 * 3 = 1203 keystrokes

**Subproblem 4: Calculate the total keystrokes**

* Total keystrokes = keystrokes for one-digit numbers + keystrokes for two-digit numbers + keystrokes for three-digit numbers
* Total keystrokes = 9 + 180 + 1203 = 1392 keystrokes


**Final Answer:**  A total of 1392 keystrokes are needed to type the numbers from 1 to 500.

this result will be better than normal prompting, now lets see what will be the answer if we dont do `COT`

In [39]:
system_prompt = """
Be a helpful assistant and answer the user's question.
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{query}"),
])

pipeline = prompt_template | llm

In [40]:
result = pipeline.invoke({"query": query}).content
display(Markdown(result))

Let's break this down:

* **Numbers 1-9:**  Each number requires one keystroke, totaling 9 keystrokes.
* **Numbers 10-99:** Each number requires two keystrokes, and there are 90 numbers in this range (99 - 10 + 1 = 90). This totals 90 * 2 = 180 keystrokes.
* **Numbers 100-999:** Each number requires three keystrokes.  We're only interested in 100-500, which is 401 numbers (500 - 100 + 1 = 401). This totals 401 * 3 = 1203 keystrokes.


Adding those together: 9 + 180 + 1203 = **1392 keystrokes**

Though we got the correct answer, for some complex questions model may hallucinate and give wrong answers but using COT sometimes we get better result as that how human also give better answer when we solve step by step ~ giving more time/breaking into smaller parts